In [2]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import sklearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import confusion_matrix, accuracy_score

from DataGenerator import *
from Competition import *
from config import *

In [3]:
BN_EO = initialize_BN()
dataset = BN_EO.simulate(n_samples=50000, seed=SEED)

  0%|          | 0/8 [00:00<?, ?it/s]

In [4]:
dataset

,CGPA,INTERN,JOB,SEX,SAT,COLLEGE,SES,SCHOOL
0,2,1,1,1,2,1,2,1
1,2,1,0,1,1,0,2,2
2,2,1,0,1,1,0,0,1
3,0,0,0,1,2,0,1,1
4,0,0,0,0,2,0,2,1
...,...,...,...,...,...,...,...,...
49995,1,0,0,0,0,0,0,0
49996,1,1,1,0,1,1,2,1
49997,1,1,1,0,0,1,1,0
49998,2,1,1,1,1,1,2,1


In [5]:
C1_label = 'COLLEGE'
C1_features = ['SES', 'SEX', 'SAT', 'SCHOOL']
C1 = Competition(dataset, C1_features, C1_label, ['SEX', 'SES'], [0,0])

In [6]:
C2_label = 'INTERN'
C2_features = ['SES', 'SEX', 'SAT', 'SCHOOL','COLLEGE','CGPA']
C2 = Competition(dataset, C2_features, C2_label, ['SEX', 'SES'], [0,0])

In [7]:
C3_label = 'JOB'
C3_features = ['SES', 'SEX', 'SAT', 'SCHOOL','COLLEGE','CGPA','INTERN']
C3 = Competition(dataset, C2_features, C2_label, ['SEX', 'SES'], [0,0])

In [8]:
X_train, y_train, X_test, y_test, X_val, y_val = C1.create_train_test_val_split(SEED=44)
X_train.shape, X_test.shape, X_val.shape

((30000, 4), (10000, 4), (10000, 4))

In [11]:
__ = C2.create_train_test_val_split(SEED=44)
__ = C3.create_train_test_val_split(SEED=44)

In [12]:
for i in C1.test_groups.keys():
    print(i, C1.test_groups[i].shape[0]/C1.X_test.shape[0])

SEX_SES_priv 0.1191
SEX_SES_dis 0.313
SEX_priv 0.6057
SEX_dis 0.3943
SES_priv 0.2004
SES_dis 0.7996


In [13]:
best_models = {}
#for model_name in model_specs.keys():
for model_name in ['lr']:
    print(model_name)
    search = GridSearchCV(model_specs[model_name]["base_model"], model_specs[model_name]["params"], cv=5, n_jobs=-1, verbose=True)
    model = search.fit(X_train, y_train)
    best_models[model_name] = model.best_estimator_
    
best_models

lr
Fitting 5 folds for each of 25 candidates, totalling 125 fits


{'lr': LogisticRegression(C=1, max_iter=200, random_state=111, solver='newton-cg')}

In [14]:
best_model = best_models['lr']
res_df = C1.fit_base_model(best_model)

In [17]:
C1.compute_predictive_metrics()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names seen at fit time, yet now missing:
- CGPA
- COLLEGE

  warnings.warn(message, FutureWarning)


ValueError: X has 4 features, but LogisticRegression is expecting 6 features as input.

In [15]:
res2_df = C2.fit_base_model(best_model)

In [ ]:
res3_df = C3.fit_base_model(best_model)

In [ ]:
C1.X_train

In [ ]:
def sigmoid(x):
    return np.array([_sigmoid_function(value) for value in x])

def _sigmoid_function(x):
    if x >= 0:
        z = np.exp(-x)
        return 1 / (1 + z)
    else:
        z = np.exp(x)
        return z / (1 + z)

In [ ]:
def mse(y_true, y_pred, sample_weights=None):
    return np.average((y_true - y_pred) ** 2, axis=0, weights=sample_weights)

In [ ]:
def cross_entropy(y_true, y_pred):
    # binary cross entropy
    y_zero_loss = y_true * np.log(y_pred + 1e-9)
    y_one_loss = (1-y_true) * np.log(1 - y_pred + 1e-9)
    return -np.mean(y_zero_loss + y_one_loss)

In [ ]:
def C_forward_linear(X, Y):
    # SES, SEX, SAT, SCHOOL
    # 0 is priv
    beta = [-0.5, -0.2, -0.6, -0.1]
    beta_y = -0.8
    return sigmoid(np.matmul(X,beta)+beta_y*Y)

In [ ]:
class CustomLinearModel:
    """
    Linear model: Y = XB, fit by minimizing the provided loss_function
    with custom regularization
    """
    def __init__(self, loss_function=cross_entropy, C_forward=C_forward_linear,
                 X=None, Y=None, sample_weights=None, beta_init=None, 
                 regularization=0.0015):
        self.regularization = regularization
        self.beta = None
        self.loss_function = loss_function
        self.C_forward_fuction = C_forward
        self.sample_weights = sample_weights
        self.beta_init = beta_init
        
        self.model_error__ = []
        self.forward_error__ = []
        self.X = X
        self.Y = Y
            
    def predict(self, X):
        prediction = sigmoid(np.matmul(X, self.beta))
        return(prediction)

    def model_error(self):
        error = self.loss_function(
            self.predict(self.X), self.Y
        )
        #print("model error: ", error)
        self.model_error__.append(error)
        return(error)
    
    def C2_error(self):
        error = np.linalg.norm((self.C_forward_fuction(self.X, self.predict(self.X))), ord=1)
        #print("forward error: ", error)
        self.forward_error__.append(error)
        return(error)
    
    def custom_foward_loss(self, beta):
        self.beta = beta
        return (self.model_error() + (self.regularization*self.C2_error()))
    
    def l2_regularized_loss(self, beta):
        self.beta = beta
        return(self.model_error() + \
               sum(self.regularization*np.array(self.beta)**2))
    
    def plot_train_dynamics(self, scale_factor):
        plt.figure(figsize=(10,5))
        plt.plot(range(len(self.forward_error__)), [i * scale_factor for i in self.forward_error__], label='forward_err')
        plt.plot(range(len(self.forward_error__)), self.model_error__, label='model_err')
        plt.legend()
        plt.show()
    
    def fit(self, maxiter=250):        
        # Initialize beta estimates (you may need to normalize
        # your data and choose smarter initialization values
        # depending on the shape of your loss function)
        if type(self.beta_init)==type(None):
            # set beta_init = 1 for every feature
            self.beta_init = np.array([0.1]*self.X.shape[1])
        else: 
            # Use provided initial values
            pass
            
        if self.beta!=None and all(self.beta_init == self.beta):
            print("Model already fit once; continuing fit with more iterations.")
            
        res = minimize(self.custom_foward_loss, self.beta_init,
                       method='BFGS', options={'maxiter': maxiter})
        self.beta = res.x
        self.beta_init = self.beta

In [1]:
c1_fair = CustomLinearModel(X=C1.X_train.values, Y=C1.y_train.values, regularization=0.02)

NameError: name 'CustomLinearModel' is not defined

In [ ]:
#lambda_range = [0, 0.0001, 0.001, 0.005, 0.01, 0.02, 0.05, 0.075, 0.1, 0.15]
lambda_range = [0, 0.0001, 0.001, 0.005, 0.01, 0.02, 0.05, 0.075, 0.1, 0.2, 1]
acc = []
for lambda_ in lambda_range:
    print(lambda_)
    c1_fair = CustomLinearModel(X=C1.X_train.values, Y=C1.y_train.values, regularization=lambda_)
    c1_fair.fit(maxiter=250)
    c1_fair_preds = np.round(c1_fair.predict(C1.X_test.values))
    acc.append(accuracy_score(C1.y_test.values, c1_fair_preds))
    c1_fair.plot_train_dynamics(0.1*lambda_)
    
print(c1_fair.beta)

In [ ]:
plt.plot(lambda_range, acc)
plt.show()